In [101]:
from gensim import corpora, models, similarities
from gensim import corpora
import gensim
import pandas as pd
import numpy as np
import os
from glob import glob
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import sys
from sklearn import linear_model
sys.path.append(os.getcwd())
import matplotlib.pyplot as plt

In [2]:
judge2count_bigram = pd.read_pickle('../rawdata/judge2count_bigram.pkl')

In [14]:
word2id = pd.read_pickle('../rawdata/word2id-2.pkl')

In [55]:
judge2count_bigram['dict'] = Counter(word2id.values())

In [56]:
v = DictVectorizer(sparse=False)
judge2count_bigram_matrix = \
    v.fit_transform(list(judge2count_bigram.values()))

In [58]:
judge2count_bigram_corpus = \
    gensim.matutils.Dense2Corpus(judge2count_bigram_matrix.T)

## bigram feature importance rank

In [17]:
data = pd.read_csv('../Holger_train.csv',decimal=',',index_col=0)
test = pd.read_csv('../Holger_test.csv',decimal=',',index_col=0)
data.head()

,trial,monsex,newcit,nocounts,judgeid,malejudge,judge_yearsonbench,demean_logsenttot,year1,year2,...,cr13,cr14,cr15,cr16,cr17,cr18,cr19,cr20,cr21,cr22
199761,0.0,0.0,0.0,2.0,"MOLLOY, DONALD W.",1.0,15.0,1.2169876098632812,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199371,1.0,0.0,0.0,1.0,"SHANSTROM, JACK D.",1.0,20.0,-0.3454446792602539,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146029,0.0,0.0,0.0,2.0,"SCOTT, JEANNE E.",0.0,11.0,0.28284311294555664,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219420,0.0,0.0,0.0,3.0,"WHITE, RONALD A.",1.0,6.0,1.1730012893676758,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174779,0.0,0.0,0.0,1.0,"HOVLAND, DANIEL L.",1.0,7.0,0.1704235076904297,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [22]:
drops = ['judgeid', 'demean_logsenttot','cr22','malejudge']
for i in range(10):
    drops.append('year' + str(i+1))
for i in range(8):
    drops.append('race' + str(i+1))

In [28]:
X_train = data.drop(drops, axis = 1)
X_train['intercept'] = 1
Y_train = np.array(data['demean_logsenttot'], dtype=np.float32)
X_test = test.drop(drops, axis = 1)
X_test['intercept'] = 1
Y_test = np.array(test['demean_logsenttot'] ,dtype=np.float32)

In [29]:
## base line
regr = linear_model.LinearRegression(fit_intercept = False)
regr.fit(X_train, Y_train)
print("Mean squared error: %.2f"
      % np.mean((regr.predict(X_test) - Y_test) ** 2))

Mean squared error: 1.12


## test all bigrams one by one

In [61]:
predictions = []
improvements = []
i = 0
for k, v in word2id.items():
    feature = pd.DataFrame(judge2count_bigram_matrix[:, v-1], columns=[k])
    feature['judgeid'] = np.array(list(judge2count_bigram.keys()))
    Train_data = pd.merge(data, feature, how='inner', on='judgeid')
    X_train = Train_data.drop(drops, axis = 1)
    X_train['intercept'] = 1
    Y_train = np.array(Train_data['demean_logsenttot'], dtype=np.float32)
    
    Test_data = pd.merge(test, feature, how='inner', on='judgeid')
    X_test = Test_data.drop(drops, axis = 1)
    X_test['intercept'] = 1
    Y_test = np.array(Test_data['demean_logsenttot'] ,dtype=np.float32)
    
    
    regr.fit(X_train, Y_train)
    accuracy = np.mean((regr.predict(X_test) - Y_test) ** 2)
    predictions.append(accuracy)
    improvements.append((accuracy - 1.12)/ accuracy)
    i += 1
    if(i % 1000 ==0 ):
        print(i)

KeyboardInterrupt: 

In [88]:
words = np.array(list(word2id.keys()))

In [92]:
predict_df = pd.DataFrame({"words" : words[:len(predictions)], 
                           "predictions" : np.array(predictions), 
                           "improvements" : -1*np.array(improvements)
                          })

In [98]:
predict_df = predict_df.sort_values(['improvements'],ascending=False)
predict_df.to_csv('../result/bigram_prediction_on_Hogler_data.csv')

In [99]:
predict_df

,improvements,predictions,words
490,0.003146,1.116488,type_prefer
2327,0.003086,1.116555,central_author
1313,0.003024,1.116623,introductori
896,0.003006,1.116644,final_consist
1517,0.002977,1.116676,inform_competit
2149,0.002952,1.116703,valid_measur
2462,0.002905,1.116756,heavili_affect
2033,0.002903,1.116758,liabil_industri
1166,0.002897,1.116764,mean_shift
2506,0.002861,1.116805,salient_issu


In [ ]:
plt.plot(Y_train, regr.predict(X_train), color='blue')
plt.show()